In [1]:
import json

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)

max_len_article = config["max_len_article"]
max_len_abstract = config["max_len_abstract"]
x_voc = config["x_voc"]
y_voc = config["y_voc"]
latent_dim = config["latent_dim"]
embedding_dim = config["embedding_dim"]

print("Configurations loaded successfully.")

Configurations loaded successfully.


In [3]:
print("max_len_article:", max_len_article)
print("max_len_abstract:", max_len_abstract)
print("x_voc:", x_voc)
print("y_voc:", y_voc)
print("latent_dim:", latent_dim)
print("embedding_dim:", embedding_dim)

max_len_article: 5000
max_len_abstract: 200
x_voc: 19582
y_voc: 4042
latent_dim: 300
embedding_dim: 200


In [4]:
import pickle

In [5]:
with open('x_tokenizer.pkl', 'rb') as handle:
    x_tokenizer = pickle.load(handle)

with open('y_tokenizer.pkl', 'rb') as handle:
    y_tokenizer = pickle.load(handle)

In [6]:
import numpy as np
import pandas as pd

In [7]:
x_tr = np.load('x_tr.npy')
y_tr = np.load('y_tr.npy')
x_val = np.load('x_val.npy')
y_val = np.load('y_val.npy')

print("Données chargées avec succès.")

Données chargées avec succès.


In [8]:
print("x_tr shape:", x_tr.shape)
print("y_tr shape:", y_tr.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)

x_tr shape: (876, 5000)
y_tr shape: (876, 200)
x_val shape: (98, 5000)
y_val shape: (98, 200)


In [9]:
from tensorflow.keras.models import load_model

model_path = 'seq2seq2_model.keras'

model = load_model(model_path)

print("Modèle chargé avec succès.")


Modèle chargé avec succès.


In [10]:
model = load_model(model_path)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 5000)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 5000, 200)         │       3,916,400 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ [(None, 5000, 300),       │         601,200 │ embedding[0][0]            │
│                               │ (None, 300), (None, 300)] │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_1 (LSTM)                 │ [(None, 5000, 300),       │         721,200 │ lstm[0][0]                 │
│                               │ (None, 300), (None, 300)] │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_1 (Embedding)       │ (None, None, 200)         │         808,400 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_2 (LSTM)                 │ [(None, 5000, 300),       │         721,200 │ lstm_1[0][0]               │
│                               │ (None, 300), (None, 300)] │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_3 (LSTM)                 │ [(None, None, 300),       │         601,200 │ embedding_1[0][0],         │
│                               │ (None, 300), (None, 300)] │                 │ lstm_2[0][1], lstm_2[0][2] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ time_distributed              │ (None, None, 4042)        │       1,216,642 │ lstm_3[0][0]               │
│ (TimeDistributed)             │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,758,728 (98.26 MB)

 Trainable params: 8,586,242 (32.75 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,172,486 (65.51 MB)

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def decode_sequence(input_seq, model, y_tokenizer):
    reverse_y_word_index = y_tokenizer.index_word
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = y_tokenizer.word_index['start'] 

    decoded_sentence = ''
    stop_condition = False
    max_len_abstract = config["max_len_abstract"]

    while not stop_condition:
        output_tokens = model.predict([input_seq, target_seq])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_y_word_index.get(sampled_token_index, '')

        if sampled_word and sampled_word != 'end':
            decoded_sentence += ' ' + sampled_word

        if sampled_word == 'end' or len(decoded_sentence.split()) >= max_len_abstract:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

    return decoded_sentence


input_seq = x_tr[0].reshape(1, -1)
predicted_summary = decode_sequence(input_seq, model, y_tokenizer)


print("Predicted summary:")
print(predicted_summary)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [11]:
def decode_sequence(input_seq, model, y_tokenizer, config):
    reverse_y_word_index = y_tokenizer.index_word

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = y_tokenizer.word_index['sostok']

    decoded_sentence = ''
    stop_condition = False
    max_len_abstract = config["max_len_abstract"]  

    while not stop_condition:
        output_tokens = model.predict([input_seq, target_seq])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_y_word_index.get(sampled_token_index, '')

        if sampled_word and sampled_word not in ['eostok', 'end']:
            decoded_sentence += ' ' + sampled_word

        if sampled_word in ['eostok', 'end'] or len(decoded_sentence.split()) >= max_len_abstract:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

    return decoded_sentence.strip()  


input_seq = x_tr[0].reshape(1, -1)  
predicted_summary = decode_sequence(input_seq, model, y_tokenizer, config)

print("Predicted summary:")
print(predicted_summary)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 